In [11]:
import os

In [12]:
os.chdir("../")

In [13]:
%pwd

'c:\\Users\\Jagadeep\\Desktop\\ml playground\\projects'

In [1]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path


In [2]:
from winequality.constants import *
from winequality.utils.common import read_yaml, create_directories

In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [4]:
import os
import urllib.request as request
import zipfile
from winequality import logger
from winequality.utils.common import get_size

In [5]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [6]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-21 23:08:23,380: INFO: yaml file: c:\users\jagadeep\desktop\ml playground\projects\wine quality end to end\config\config.yaml loaded succesfully]
[2025-06-21 23:08:23,382: INFO: yaml file: c:\users\jagadeep\desktop\ml playground\projects\wine quality end to end\params.yaml loaded succesfully]
[2025-06-21 23:08:23,385: INFO: yaml file: c:\users\jagadeep\desktop\ml playground\projects\wine quality end to end\schema.yaml loaded succesfully]
[2025-06-21 23:08:23,386: INFO: created directory at :artifacts_root]
[2025-06-21 23:08:23,387: INFO: created directory at :artifacts/data_ingestion]
[2025-06-21 23:08:24,860: INFO: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 21497
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9dd794b9fc858075b432f971f1a022cc166132cedc98dd878a9d24c5afe9f839"
Strict-Transport-Security: max-age=31536000
X-Conte